In [0]:
# wget https://raw.githubusercontent.com/jbrownlee/Datasets/master/wheat-seeds.csv

In [2]:
#We will not use pandas to load dataframe this is only for viewing the dataset
import pandas as pd
df=pd.read_csv('wheat-seeds.csv',header=None)
df[7].unique()
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210 entries, 0 to 209
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       210 non-null    float64
 1   1       210 non-null    float64
 2   2       210 non-null    float64
 3   3       210 non-null    float64
 4   4       210 non-null    float64
 5   5       210 non-null    float64
 6   6       210 non-null    float64
 7   7       210 non-null    int64  
dtypes: float64(7), int64(1)
memory usage: 13.2 KB
None


This Tutorial has been broken down into 6 parts...
1. Initialise network
2. Forward Propagate
3. Back Propagate Error
4. Train Network
5. Predict
6. Evaluate Prediction model

# 1. Initialise Network

The Network is organised into layers and each layer is made of various neurons. The input layer is just a row of our dataset. The first real layer is the hidden layer. It is followed by the output layer which has one neuron for each class value.
We will organise layers as arrays of dictionaries and the whole network as an array of layers.

In [0]:
def initialize_network(n_inputs,n_hidden,n_outputs):
    network=list()
    hidden_layer = [{'weights':[random() for i in range(n_inputs+1)]} for i in range(n_hidden)]
    network.append(hidden_layer)
    output_layer = [{'weights':[random() for i in range(n_hidden+1)]} for i in range(n_outputs)]
    network.append(output_layer)
    return network

In [2]:
from random import random
network = initialize_network(2,1,2)
for layer in network:
    print(layer)

[{'weights': [0.16806403492875022, 0.2817280053097815, 0.9126240487200058]}]
[{'weights': [0.39125701682384595, 0.8881015975640839]}, {'weights': [0.4697303308232721, 0.32094778001391355]}]


# 2. Forward Propagate

We can calculate output from a neural network by propagating an input signal through the network until the output layer outputs its values.
We call this forward propagation.<br>
We can break forward propagation into 3 steps :
1. Neuron Activation
2. Neuron Transfer
3. Forward Propagation

### 2.1. Neuron Activation
Neuron activation is calculated as the weighed sum of inputs just like Linear Regression.<br>
<code>activation=sum(weight_i*input_i)+bias</code><br>
The activate function assumes that the bias is the last weight in the list of weights.

In [0]:
def activate(weights,inputs):
    activation=weights[-1]
    for i in range(len(weights)-1):
       activation+=weights[i]*inputs[i]
    return activation 

### 2.2. Neuron Transfer
Once neuron is activated we need to transfer the activation to see what the output is<br>
Different transfer functions can be used but it is traditional to use sigmoid activation function but you can also use hyperbolic tangent or Relu Activation.<br>
**Sigmoid activation**<br>
<code>output = 1/(1+e^-activation)</code>

In [0]:
from math import exp
def transfer(activation):
    return 1/(1+exp(-activation))

### 2.3. Forward Propagation
We propagate the data from input layer calculating the output of each neuron. All of the output of one layer becomes input to next layer neurons.

In [0]:
def forward_propagate(network,row):
    inputs=row
    for layer in network:
        new_inputs=[]
        for neuron in layer:
            activation=activate(neuron['weights'],inputs)
            neuron['output']=transfer(activation)
            new_inputs.append(neuron['output'])
        inputs=new_inputs
    return inputs

In [8]:
print(forward_propagate(network,[1,0,None]))

[0.6594075188749586, 0.6870411377882268]


# 3. Back Propagation
Error is calculated between the actual outputs and the output forward propagated from the network
<br>This part is broken down into 2 sections:
<br>
1. Transfer Derivative
2. Error Backpropagation

### 3.1. Transfer Derivative
Given an output from a neuron we need to calculate its slope.<br>
We are using **Sigmoid activation function** the derivative of which can be calculated as<br>
<code>derivative = output * (1.0 - output)</code>

In [0]:
def transfer_derivative(output):
    return output*(1.0-output)

### 3.2. Error Backpropagation
The first step is to calculate error for each output neuron<br>
The error for a given neuron can be calculated as follows,<br>
<code>error = (expected-output) * transfer_derivative(output)</code><br>
This error calculation is used for calculating error in the output layer<br>
The error signal for a neuron in hidden layer is little bit more complicated. It is calculated as weighted error of each layer in output neuron<br>
<code>error = (weight_k * error_j) * transfer_derivative(output)</code><br>
Where **error_j** is the error signal from the j-th neuron in the output layer, **weight_k** is the weight that connects k-th neuron to the current neuron.

In [0]:
def backward_propagate_error(network,expected):
    for i in reversed(range(len(network))):
        layer=network[i]
        errors=list()
        if i!=len(network)-1:
            for j in range(len(layer)):
                error=0.0
                for neuron in network[i+1]:
                    error+=neuron['weights'][j]*neuron['delta']
                errors.append(error)
        else:
            for j in range(len(layer)):
                neuron=layer[j]
                errors.append(expected[j]-neuron['output'])
        for j in range(len(layer)):
            neuron=layer[j]
            neuron['delta']=errors[j]*transfer_derivative(neuron['output'])

In [0]:
def back_propagation(train, test, l_rate, n_epoch, n_hidden):
	n_inputs = len(train[0]) - 1
	n_outputs = len(set([row[-1] for row in train]))
	network = initialize_network(n_inputs, n_hidden, n_outputs)
	train_network(network, train, l_rate, n_epoch, n_outputs)
	predictions = list()
	for row in test:
		prediction = predict(network, row)
		predictions.append(prediction)
	return(predictions)

In [11]:
#Test back poropagation
network = [[{'output': 0.7105668883115941, 'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}],
		[{'output': 0.6213859615555266, 'weights': [0.2550690257394217, 0.49543508709194095]}, {'output': 0.6573693455986976, 'weights': [0.4494910647887381, 0.651592972722763]}]]
expected = [0, 1]
backward_propagate_error(network, expected)
for layer in network:
	print(layer)

[{'output': 0.7105668883115941, 'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614], 'delta': -0.0005348048046610517}]
[{'output': 0.6213859615555266, 'weights': [0.2550690257394217, 0.49543508709194095], 'delta': -0.14619064683582808}, {'output': 0.6573693455986976, 'weights': [0.4494910647887381, 0.651592972722763], 'delta': 0.0771723774346327}]


# 4. Train Network
Network is trained using stochatic gradient descent
<br>
This involves multiple iterations of exposing a training dataset for each row of dataset, forward propagating the error and backward propagating the error and updating the network weights<br>
This part is broken down into two sections
1. Update Weights
2. Train Network

### 4.1. Update Weights
Once error is calculated for each neuron using back propagation, they can be used to update weights<br>
Network weights are updated as follows,<br>
<code>weight += learning_rate * error * input</code><br>
where **weight** is the given weight, **learning_rate** is a parameter, **error** is the error calculated by back propagation, and **input** is the input value that caused the error.<br>
The same procedure can be used for updating the bias weights except that there is no input value, or input is the fixed value of 1.0

In [0]:
def update_weights(network,row,l_rate):
    for i in range(len(network)):
        inputs = row[:-1]
        if i != 0:
            inputs=[neuron['output'] for neuron in network[i-1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j]+=l_rate*neuron['delta']*inputs[j]
            neuron['weights'][-1]+=l_rate*neuron['delta']

### 4.2. Train Network

In [0]:
def train_network(network,train,l_rate,n_epochs,n_outputs):
    for epoch in range(n_epochs):
        for row in train:
            outputs = forward_propagate(network, row)
            expected = [0 for i in range(n_outputs)]
            expected[row[-1]] = 1
            backward_propagate_error(network, expected)
            update_weights(network, row, l_rate)

# 5. Predict

In [0]:
def predict(network,row):
    outputs = forward_propagate(network,row)
    return outputs.index(max(outputs))

In [0]:
from csv import reader
from random import random
from random import randrange
from math import exp

def str_column_to_float(dataset, column):
	for row in dataset:
		row[column] = float(row[column].strip())
 
# Convert string column to integer
def str_column_to_int(dataset, column):
	class_values = [row[column] for row in dataset]
	unique = set(class_values)
	lookup = dict()
	for i, value in enumerate(unique):
		lookup[value] = i
	for row in dataset:
		row[column] = lookup[row[column]]
	return lookup
 
# Find the min and max values for each column
def dataset_minmax(dataset):
	minmax = list()
	stats = [[min(column), max(column)] for column in zip(*dataset)]
	return stats
 
# Rescale dataset columns to the range 0-1
def normalize_dataset(dataset, minmax):
	for row in dataset:
		for i in range(len(row)-1):
			row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])
 
# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
	dataset_split = list()
	dataset_copy = list(dataset)
	fold_size = int(len(dataset) / n_folds)
	for i in range(n_folds):
		fold = list()
		while len(fold) < fold_size:
			index = randrange(len(dataset_copy))
			fold.append(dataset_copy.pop(index))
		dataset_split.append(fold)
	return dataset_split
 
# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
	correct = 0
	for i in range(len(actual)):
		if actual[i] == predicted[i]:
			correct += 1
	return correct / float(len(actual)) * 100.0
 
# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
	folds = cross_validation_split(dataset, n_folds)
	scores = list()
	for fold in folds:
		train_set = list(folds)
		train_set.remove(fold)
		train_set = sum(train_set, [])
		test_set = list()
		for row in fold:
			row_copy = list(row)
			test_set.append(row_copy)
			row_copy[-1] = None
		predicted = algorithm(train_set, test_set, *args)
		actual = [row[-1] for row in fold]
		accuracy = accuracy_metric(actual, predicted)
		scores.append(accuracy)
	return scores

In [0]:
def load_csv(filename):
    dataset=list()
    with open(filename,'r') as file:
        rows = reader(file)
        for row in rows:
            if not row:
                continue
            dataset.append(row)
        return dataset 

In [16]:
dataset = load_csv('wheat-seeds.csv')
for i in range(len(dataset[0])-1):
    str_column_to_float(dataset,i)
str_column_to_int(dataset,len(dataset[0])-1)
minmax = dataset_minmax(dataset)
normalize_dataset(dataset,minmax)
scores = evaluate_algorithm(dataset,back_propagation,5,0.3,500,5)
print(scores)
print('Mean Accuracy ',sum(scores)/len(scores))

[97.61904761904762, 92.85714285714286, 90.47619047619048, 90.47619047619048, 97.61904761904762]
Mean Accuracy  93.80952380952381
